###	1.	Install Necessary Libraries:
Ensure that you have the necessary libraries installed. You can install PyTorch-Geometric and its dependencies using pip if you haven’t already:

In [50]:
#!pip install torch torch-geometric

###	2.	Load and Prepare the Dataset:
We’ll use the Cora dataset, which is a standard citation network dataset. The nodes represent documents, and the edges represent citations between them.

In [51]:
from torch_geometric.transforms import RandomLinkSplit
from torch_geometric import transforms
import torch
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import train_test_split_edges

# Load the Cora dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')

# Use the first graph in the dataset
data = dataset[0]

# Split the edges into training, validation, and test sets
transform = RandomLinkSplit(is_undirected=True, key="edge_label", split_labels=True)

train_data, val_data, test_data = transform(dataset[0])

###	3.	Define the VGAE Model:
We need to define the VGAE model, which consists of an encoder that maps the input features into a latent space. The encoder is typically implemented using Graph Convolutional Networks (GCNs).

In [52]:
from torch_geometric.nn import VGAE, GCNConv
class GCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels)
        self.conv_mu = GCNConv(2 * out_channels, out_channels)
        self.conv_logstd = GCNConv(2 * out_channels, out_channels)
    
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        mu = self.conv_mu(x, edge_index)
        logstd = self.conv_logstd(x, edge_index)
        return mu, logstd

# Initialize the VGAE model
out_channels = 16
model = VGAE(GCNEncoder(dataset.num_features, out_channels))

###	4.	Training the Model:
The model is trained by optimizing a loss function that combines a reconstruction loss (which ensures the graph is reconstructed correctly) and a KL divergence loss (which regularizes the latent space).

In [53]:
# Define optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training loop
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)
    loss = model.recon_loss(z, train_data.edge_index)
    loss = loss + (1 / train_data.num_nodes) * model.kl_loss()
    loss.backward()
    optimizer.step()
    return loss.item()

# Train the model for 200 epochs
for epoch in range(200):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

Epoch: 000, Loss: 3.5240
Epoch: 001, Loss: 2.8785
Epoch: 002, Loss: 2.5037
Epoch: 003, Loss: 2.2285
Epoch: 004, Loss: 1.9997
Epoch: 005, Loss: 1.7829
Epoch: 006, Loss: 1.6121
Epoch: 007, Loss: 1.4911
Epoch: 008, Loss: 1.4171
Epoch: 009, Loss: 1.3659
Epoch: 010, Loss: 1.3272
Epoch: 011, Loss: 1.3136
Epoch: 012, Loss: 1.2871
Epoch: 013, Loss: 1.2628
Epoch: 014, Loss: 1.2442
Epoch: 015, Loss: 1.1994
Epoch: 016, Loss: 1.1672
Epoch: 017, Loss: 1.1505
Epoch: 018, Loss: 1.1083
Epoch: 019, Loss: 1.0933
Epoch: 020, Loss: 1.0909
Epoch: 021, Loss: 1.0823
Epoch: 022, Loss: 1.0713
Epoch: 023, Loss: 1.0498
Epoch: 024, Loss: 1.0218
Epoch: 025, Loss: 1.0118
Epoch: 026, Loss: 1.0059
Epoch: 027, Loss: 1.0100
Epoch: 028, Loss: 1.0046
Epoch: 029, Loss: 0.9797
Epoch: 030, Loss: 0.9950
Epoch: 031, Loss: 0.9810
Epoch: 032, Loss: 0.9715
Epoch: 033, Loss: 0.9756
Epoch: 034, Loss: 0.9647
Epoch: 035, Loss: 0.9717
Epoch: 036, Loss: 0.9670
Epoch: 037, Loss: 0.9609
Epoch: 038, Loss: 0.9428
Epoch: 039, Loss: 0.9392


###	5.	Evaluate the Model:
After training, we can evaluate the model’s performance on the test set. The evaluation typically involves predicting the probability of edges (links) between nodes and comparing them to the true test edges.

In [54]:
# Test the model
from sklearn.metrics import roc_auc_score, average_precision_score
# Test the model
def test(data):
    model.eval()
    with torch.no_grad():
        z = model.encode(data.x, data.edge_index)
        pos_pred = model.decode(z, data.pos_edge_label_index).cpu().numpy()
        neg_pred = model.decode(z, data.neg_edge_label_index).cpu().numpy()

        y_pred = torch.cat([torch.tensor(pos_pred), torch.tensor(neg_pred)])
        y_true = torch.cat([torch.ones(pos_pred.shape[0]), torch.zeros(neg_pred.shape[0])])

        roc_auc = roc_auc_score(y_true, y_pred)
        ap_score = average_precision_score(y_true, y_pred)

        return roc_auc, ap_score

roc_auc, ap_score = test(test_data)
print(f'ROC AUC Score: {roc_auc:.4f}, Average Precision Score: {ap_score:.4f}')

ROC AUC Score: 0.9009, Average Precision Score: 0.9086
